In [15]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pydrake.geometry import StartMeshcat

from iiwa_batter import PACKAGE_ROOT, CONTROL_DT, PITCH_DT, CONTACT_DT, NUM_JOINTS
from iiwa_batter.robot_constraints.get_joint_constraints import JOINT_CONSTRAINTS
from iiwa_batter.physics import find_ball_initial_velocity, PITCH_START_POSITION
from iiwa_batter.save_load import load_trajectory

from iiwa_batter.swing_optimization.full_trajectory import run_full_trajectory, setup_simulator, make_torque_trajectory

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [ ]:
# Load control vector
trajectory_settings = {
    "robot": "kr6r900",
    "pitch_speed_mph": 90,
    "target_position": [0, 0, 0.6],
}
optimization_name = "naive_full_trajectory"

best_control_vector, _, _, _ = load_trajectory(trajectory_settings, optimization_name)

simulator, station = setup_simulator(dt=CONTACT_DT, meshcat=meshcat)
ball_initial_velocity, time_of_flight = find_ball_initial_velocity(trajectory_settings["pitch_speed_mph"], 
                                                         trajectory_settings["target_position"])
trajectory_timesteps = np.arange(0, time_of_flight, CONTROL_DT)
robot_constraints = JOINT_CONSTRAINTS["kr6r900"]
torque_trajectory = make_torque_trajectory(best_control_vector, NUM_JOINTS, trajectory_timesteps)

contact_loss = run_full_trajectory(meshcat, simulator, station, best_control_vector[:NUM_JOINTS], [PITCH_START_POSITION, ball_initial_velocity], time_of_flight, robot_constraints, torque_trajectory)

simulator, station = setup_simulator(dt=PITCH_DT, meshcat=meshcat)
pitch_loss = run_full_trajectory(meshcat, simulator, station, best_control_vector[:NUM_JOINTS], [PITCH_START_POSITION, ball_initial_velocity], time_of_flight, robot_constraints, torque_trajectory)

print(f"Contact dt loss: {contact_loss}")
print(f"Pitch dt loss: {pitch_loss}")
print(f"Difference: {contact_loss - pitch_loss}")

Contact dt loss: -11.145134498196654
Pitch dt loss: -11.174541770522898
Difference: 0.029407272326244183
